In [3]:
import time
import os
import random
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import numpy as np
import scipy
import seaborn as sns
import copy
from pandas import concat, DataFrame
from sklearn.linear_model import RANSACRegressor, LinearRegression, TheilSenRegressor
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_absolute_error, median_absolute_error, r2_score
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from pandas_datareader import data as pdr
from sklearn.neural_network import MLPRegressor
import zipfile

In [10]:
# get filenames
path = os.getcwd()
print(path)

file_names = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
print(file_names)

C:\Users\MyPC\Desktop\projects\ML_Course\week3\Fraud-Detection
['Fraud Detection.ipynb', 'ieee-fraud-detection.zip', 'README.md']


In [6]:
# unzip dataset zip file
local_zip = '{}'.format(file_names[1])
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('{}\dataset'.format(path))
zip_ref.close()

In [11]:
dir_files = [f for f in os.listdir(path) if os.path.join(path, f)]
print(dir_files)

['.git', '.ipynb_checkpoints', 'dataset', 'Fraud Detection.ipynb', 'ieee-fraud-detection.zip', 'README.md']


# Explore Data

In [19]:
data_path = 'dataset'
train_identity = pd.read_csv('dataset/train_identity.csv')
train_transaction = pd.read_csv('dataset/train_transaction.csv')
test_identity = pd.read_csv('dataset/test_identity.csv')
test_transaction = pd.read_csv('dataset/test_transaction.csv')



,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
data = train_identity.dropna(how='all')
# print(train_identity.columns)
print(train_identity['TransactionID'])
index=train_identity.index[train_identity['TransactionID'] == 2987004].tolist()
print(index)
if (index):
    print(index)
    # get index
    #get data
    new_data = train_identity.loc[index]
    print(new_data['TransactionID'].values[0])
#     for i in new_data.columns:
#         print(new_data[i].dropna()[0:10])


0         2987004
1         2987008
2         2987010
3         2987011
4         2987016
           ...   
144228    3577521
144229    3577526
144230    3577529
144231    3577531
144232    3577534
Name: TransactionID, Length: 144233, dtype: int64
[0]
[0]
2987004


In [44]:
data = train_transaction
print(train_transaction.columns)
for i in train_transaction.columns:
    print(train_transaction[i][0:20])


Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=394)
0     2987000
1     2987001
2     2987002
3     2987003
4     2987004
5     2987005
6     2987006
7     2987007
8     2987008
9     2987009
10    2987010
11    2987011
12    2987012
13    2987013
14    2987014
15    2987015
16    2987016
17    2987017
18    2987018
19    2987019
Name: TransactionID, dtype: int64
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: isFraud, dtype: int64
0     86400
1     86401
2     86469
3     86499
4     86506
5     86510
6     86522
7     86529
8     86535
9     86536
10    86549
11    86555
12    86564
13    86585
14    86596
15    86618
16    86620
17    86668
18    8

Name: V126, dtype: float64
0      117.0
1        0.0
2        0.0
3     1758.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12      60.0
13     100.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19     517.5
Name: V127, dtype: float64
0       0.0
1       0.0
2       0.0
3     925.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19    102.5
Name: V128, dtype: float64
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
Name: V129, dtype: float64
0       0.0
1       0.0
2       0.0
3     354.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12     60.0
13    100.0
14      0.0
15      0.0


Name: V251, dtype: float64
0     NaN
1     NaN
2     NaN
3     NaN
4     1.0
5     NaN
6     NaN
7     NaN
8     1.0
9     NaN
10    1.0
11    1.0
12    NaN
13    NaN
14    NaN
15    NaN
16    1.0
17    1.0
18    NaN
19    NaN
Name: V252, dtype: float64
0     NaN
1     NaN
2     NaN
3     NaN
4     1.0
5     NaN
6     NaN
7     NaN
8     1.0
9     NaN
10    1.0
11    1.0
12    NaN
13    NaN
14    NaN
15    NaN
16    1.0
17    1.0
18    NaN
19    NaN
Name: V253, dtype: float64
0     NaN
1     NaN
2     NaN
3     NaN
4     1.0
5     NaN
6     NaN
7     NaN
8     1.0
9     NaN
10    1.0
11    1.0
12    NaN
13    NaN
14    NaN
15    NaN
16    1.0
17    1.0
18    NaN
19    NaN
Name: V254, dtype: float64
0     NaN
1     NaN
2     NaN
3     NaN
4     1.0
5     NaN
6     NaN
7     NaN
8     1.0
9     NaN
10    1.0
11    1.0
12    NaN
13    NaN
14    NaN
15    NaN
16    1.0
17    1.0
18    NaN
19    NaN
Name: V255, dtype: float64
0     NaN
1     NaN
2     NaN
3     NaN
4     1.0
5     NaN
6    

In [22]:
test_identity.head()


,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,3663586,-45.0,280290.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
1,3663588,0.0,3579.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 67.0 for android,24.0,1280x720,match_status:2,T,F,T,T,mobile,LGLS676 Build/MXB48T
2,3663597,-5.0,185210.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,...,ie 11.0 for tablet,NaN,NaN,NaN,F,T,T,F,desktop,Trident/7.0
3,3663601,-45.0,252944.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
4,3663602,-95.0,328680.0,NaN,NaN,7.0,-33.0,NaN,NaN,NaN,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,SM-G9650 Build/R16NW


In [23]:
test_transaction.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Logistic Regression